In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import pandas as pd

In [12]:
# Save landmark into array
def make_landmark_timestep(results):
    l = []
    landmark_res = results.pose_landmarks.landmark
    for idx, lm in enumerate(landmark_res):
        l.append(lm.x)
        l.append(lm.y)
        l.append(lm.z)
        l.append(lm.visibility)
    return l


In [17]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose # Mediapipe Solutions
no_of_frames = 600
cap = cv2.VideoCapture(0)
lm_list = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while len(lm_list) < no_of_frames:
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = pose.process(image)

        # Save into a list
        if results.pose_landmarks:
            lm = make_landmark_timestep(results)
            lm_list.append(lm)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 1. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [18]:
label = 'NODDING'
df = pd.DataFrame(lm_list)
df.to_csv(f"./csv_data/{label}.csv",index=False)